In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

sensor_names = ['Acc_x', 'Acc_y', 'Acc_z', 'Gyr_x', 'Gyr_y', 'Gyr_z']
# Last row of training data for train/test split
train_end_index = 3511

In [26]:
labels = np.loadtxt('labels_train_1.csv', dtype='int')
data_slice_0 = np.loadtxt(sensor_names[0] + '_train_1.csv',
                            delimiter=',')
data = np.empty((data_slice_0.shape[0], data_slice_0.shape[1],
                    len(sensor_names)))
data[:, :, 0] = data_slice_0
del data_slice_0
for sensor_index in range(1, len(sensor_names)):
    data[:, :, sensor_index] = np.loadtxt(
        sensor_names[sensor_index] + '_train_1.csv', delimiter=',')

In [27]:
train_data = data[:train_end_index+1, :, :]
train_labels = labels[:train_end_index+1]
test_data = data[train_end_index+1:, :, :]
test_labels = labels[train_end_index+1:]

In [28]:
class FirstCNN(nn.Module):
    def __init__(self, num_classes):
        """
        Parameters:
        - num_classes: Number of output classes (for classification tasks).
        """
        super(FirstCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=6, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7, 128)  # Flattened size after convolutions and pooling
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # Input x shape: (batch_size, 60, 6)
        
        # Transpose to match Conv1d's expected input: (batch_size, channels, sequence_length)
        x = x.permute(0, 2, 1)  # Shape: (batch_size, 6, 60)
        
        # Convolution + ReLU + Pooling
        x = self.pool(F.relu(self.conv1(x)))  # Shape: (batch_size, 16, 30)
        x = self.pool(F.relu(self.conv2(x)))  # Shape: (batch_size, 32, 15)
        x = self.pool(F.relu(self.conv3(x)))  # Shape: (batch_size, 64, 7)
        
        # Flatten the features
        x = x.view(x.size(0), -1)  # Shape: (batch_size, 64 * 7)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # No activation; softmax or other applied later if needed
        
        return x

In [29]:
LEARNING_RATE = 0.001
NUM_CLASSES = 4
BATCH_SIZE = 256
EPOCHS = 25

# move model to gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data_pt = torch.tensor(train_data, dtype=torch.float)
train_labels_pt = F.one_hot(torch.tensor(train_labels, dtype=torch.long) - 1,  num_classes=NUM_CLASSES).float()

training_dataset = dataset = TensorDataset(train_data_pt.to(device), train_labels_pt.to(device))
dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = FirstCNN(num_classes=NUM_CLASSES).to(device)


criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, targets in dataloader:

        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)


        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(dataloader):.4f}")

Epoch 1/25, Loss: 1.2251
Epoch 2/25, Loss: 0.8873
Epoch 3/25, Loss: 0.7283
Epoch 4/25, Loss: 0.7016
Epoch 5/25, Loss: 0.6314
Epoch 6/25, Loss: 0.6018
Epoch 7/25, Loss: 0.5573
Epoch 8/25, Loss: 0.5071
Epoch 9/25, Loss: 0.4812
Epoch 10/25, Loss: 0.5136
Epoch 11/25, Loss: 0.4857
Epoch 12/25, Loss: 0.4632
Epoch 13/25, Loss: 0.4506
Epoch 14/25, Loss: 0.4621
Epoch 15/25, Loss: 0.4280
Epoch 16/25, Loss: 0.4224
Epoch 17/25, Loss: 0.3907
Epoch 18/25, Loss: 0.3900
Epoch 19/25, Loss: 0.3904
Epoch 20/25, Loss: 0.3883
Epoch 21/25, Loss: 0.3592
Epoch 22/25, Loss: 0.3394
Epoch 23/25, Loss: 0.3338
Epoch 24/25, Loss: 0.3494
Epoch 25/25, Loss: 0.3523


In [30]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    test_data_pt = torch.tensor(test_data, dtype=torch.float)

    test_data_pt = test_data_pt.to(device)
    
    test_outputs = model(test_data_pt)

    predicted_classes = torch.argmax(test_outputs, dim=1)

In [31]:
test_labels_pt = torch.tensor(test_labels, dtype=torch.long)

# Compute micro and macro-averaged F1 scores
accuracy = accuracy_score(test_labels_pt, predicted_classes.cpu())
micro_f1 = f1_score(test_labels_pt, predicted_classes.cpu(), average='micro')
macro_f1 = f1_score(test_labels_pt, predicted_classes.cpu(), average='macro')
print(f'Accuracy: {accuracy}')
print(f'Micro-averaged F1 score: {micro_f1}')
print(f'Macro-averaged F1 score: {macro_f1}')

Accuracy: 0.04532077692760447
Micro-averaged F1 score: 0.04532077692760447
Macro-averaged F1 score: 0.04711797590771612
